In [ ]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import re
import json
import csv
import urllib.error

In [ ]:
base_url = 'https://www.mystore411.com'

In [ ]:
def retrieve_shops_first_letter(x):
    """Get all letter links from the main page"""
    html = urlopen(x)
    bs = BeautifulSoup(html, 'html.parser')
    urls = []
    new_urls=[]
    for link in bs.find('div', {'id':'breadcrumb'}).find_all('a', href=re.compile('^(/store/)((?!:).)*$')):
        if 'href' in link.attrs:
            letters = link.attrs['href']
            urls.append(letters)
            new_urls = ['{}{}'.format(base_url,i) for i in urls]
    return new_urls 

In [ ]:
shops_first_letter_links = retrieve_shops_first_letter(base_url)

In [ ]:
def retrieve_stores(y):
    """Get all links of all shops"""
    stores = []
    for each in y:
        html = urlopen(each)
        soup = BeautifulSoup(html, "lxml" )
        newnewurl = []
        for ul in soup.find_all('ol', class_='col2 d_store_list'):
            for li in ul.find_all('li'):
                a = li.find('a')
                newnewurl = a['href']
                stores.append(newnewurl)
                all_stores_links = ['{}{}'.format(base_url,i) for i in stores]
    return(all_stores_links)
    

In [ ]:
all_shops_links = retrieve_stores(shops_first_letter_links)

In [ ]:
def retrieve_Canadian_shops(w):
    """Get all links which already includes the word "Canada""""
    Canada_list = [s for s in w if "/Canada/" in s]
    return Canada_list

In [ ]:
first_Canada_list = retrieve_Canadian_shops(all_shops_links)

In [ ]:
def retrieve_others_shops(w):
    """Get all links which does not include the word "Canada""""
    stores_for_work = [s for s in w if "/Canada/" not in s]
    others_stores = [re.sub(r'\s+', '%20', x) for x in stores_for_work]
    return others_stores

In [ ]:
others_stores = retrieve_others_shops(all_shops_links)

In [ ]:
def retrieve_links_from_side_menu(x):
    """Get all links of all shops of all counties from side menu"""
    second_Canada_list = []
    for each in x:
        html = urlopen(each)
        soup = BeautifulSoup(html, "lxml")
        urlCanada = []
        for ul in soup.find_all('ul', class_='sidemenu salmon'):
            for li in ul.find_all('li'): 
                a = li.find('a')
                urlCanada = a['href'] 
                second_Canada_list.append(urlCanada)
                second_Canada_list_links = ['{}{}'.format(base_url,i) for i in second_Canada_list]
    return(second_Canada_list_links)
        

In [ ]:
side_menu_links = retrieve_links_from_side_menu(others_stores)

In [ ]:
"""Divide all side menu links on 2 lists: already Canadian and not"""

In [ ]:
second_Canada_list = retrieve_Canadian_shops(side_menu_links)

In [ ]:
third_Canada_list = first_Canada_list + second_Canada_list

In [ ]:
def retrieve_provincies(x):
    """Get all links of all provinces inside all Canadian links"""
    provinces = []
    for each in x:
        html = urlopen(each)
        soup = BeautifulSoup(html, "lxml")
        urlProv = []
        for table in soup.find_all('table', class_='table1'):
            for a in table.find_all('a'):
                urlProv = a['href']
                provinces.append(urlProv)
                provinces_list = ['{}{}'.format(base_url,i) for i in provinces]
                provinces_list_links = [re.sub(r'\s+', '%20', x) for x in provinces_list]
    return(provinces_list_links)

In [ ]:
all_provinces_links = retrieve_provincies(third_Canada_list)

In [ ]:
"""My goal is to get all links with "view" inside"""

In [ ]:
def retrieve_links_with_view_inside(x):
    """Get links that is ready for scraping"""
    view_list = [s for s in x if "/view/" in s]
    return view_list

In [ ]:
first_list_ready_for_scraping = retrieve_links_with_view_inside(all_provinces_links)

In [ ]:
def retrieve_links_for_next_step(x):
    """Get new list for one more step before scraping"""
    working_list = [s for s in x if "/view/" not in s]
    return working_list

In [ ]:
list_to_work_on = retrieve_links_for_next_step(all_provinces_links)

In [ ]:
def retrieve_view_links(x):
    """Get links with "view" from the last list of links"""
    links = []
    for each in x:
        try:
            html = urlopen(each)
            soup = BeautifulSoup(html, "lxml")
            
        except (urllib.error.HTTPError,  urllib.error.URLError, UnicodeEncodeError):
               print('Continue') 
        
        view_links = []
        for table in soup.find_all('table', class_='table1'):
            for a in table.find_all('a'):
                view_links = a['href']
                links.append(view_links)
                all_links = ['{}{}'.format(base_url,i) for i in links]
    return(all_links)

In [ ]:
second_list_ready_for_scraping = retrieve_view_links(list_to_work_on)

In [ ]:
final_list_ready_for_scraping = first_list_ready_for_scraping + second_list_ready_for_scraping

In [ ]:
def create_csv(x):
    """Create and fill out csv with necessary information"""
    stores=[]
    addresses=[]
    cities=[]
    provincies=[]
    postalcodes=[]

    for n in stores_ready_for_scraping9:
        try:
            html = urlopen(n).read().decode('utf-8')
            soup = BeautifulSoup(html, "lxml")
        except (urllib.error.HTTPError,  urllib.error.URLError, UnicodeEncodeError):
            print('Continue Scraping') 
        
    for store in soup.find('span', itemprop = 'name'):
        stores.append(store)
        
    
    address = soup.find(itemprop = 'streetAddress').get_text()
    addresses.append(address)
    
    
    city = soup.find(itemprop = 'addressLocality').get_text()
    cities.append(city)
    
    province = soup.find(itemprop = 'addressRegion').get_text()
    provincies.append(province)
    
    postalcode = soup.find(itemprop = 'postalCode').get_text()
    postalcodes.append(postalcode)
    
    with open('All_Stores.csv', 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        
        writer.writerow(("Store name", "Address", "City", "Province", "Postal Code"))
        writer.writerows(zip(stores, addresses, cities, provincies, postalcodes))    

In [ ]:
create_csv(final_list_ready_for_scraping)